## Задание - провести исследование и предобработку имеющихся данных по букмекерской компании с целью дальнейшей работы с данными
Данные по букмекерской конмпании представлены в двух файлах:  
- ***log.csv***  - данные по времени, ставкам и выигрышам по каждому пользователю
- ***user.csv*** - данные по местонахождению и email адресу по каждому пользователю

In [1]:
import pandas as pd

log = pd.read_csv('log.csv', header=None)
display(log.head(2))

users = pd.read_csv('users.csv', encoding='koi8-r', sep='\t')
display(users.head(2))

,0,1,2,3
0,Запись пользователя № - user_919,[2019-01-01 14:06:51,NaN,NaN
1,Запись пользователя № - user_973,[2019-01-01 14:51:16,NaN,NaN


,Юзверь,мейл,Гео
0,User_943,Accumanst@gmail.com,Ижевск
1,User_908,Advismowr@mail.ru,Ижевск


Подготовка к работе файла ***log.csv*** :

In [2]:
#переименование столбцов
log.columns = ['user_id', 'time', 'bet', 'win']

#заполнение пустых ячеек в столбцах "bet" и "win" нулями
log.bet = log['bet'].fillna(0)
log.win = log['win'].fillna(0)

#создание столбца "net" - как результата по текущей ставке
log['net'] = log['win'] - log['bet']

#приведение столбца user_id к удобному виду "user_***" и удаление ошибочных данных "#error"
log.user_id = log.user_id.apply(lambda x: x.replace('Запись пользователя № - ',''))
log = log[log.user_id != '#error']

#преобразование столбца "time" 
log.time = log.time.str.strip('[')
log['time'] = pd.to_datetime(log['time'])

log.sample(2)

,user_id,time,bet,win,net
154,user_977,2019-01-16 06:04:59,0.0,0.0,0.0
185,user_992,2019-01-20 09:43:39,0.0,0.0,0.0


Подготовка к работе файла ***users.csv*** :

In [3]:
#переименование столбцов
users.columns = ['user_id', 'email', 'geo']

#преобразование стобца "user_id" к одному виду с df log
users.user_id = users.user_id.apply(lambda x: x.lower())
users.sample(2)

,user_id,email,geo
96,user_901,V2artierso@mail.ru,Арзангелтск
61,user_942,Mailitry@ya.ru,Москва


Объединение 2х получившихся датафреймов по полю ***user_id*** :

In [4]:
log_us = pd.merge(log, users, how = 'inner', on = 'user_id')
log_us.sample(2)

,user_id,time,bet,win,net,email,geo
459,user_999,2019-02-02 15:28:36,0.0,0.0,0.0,Stacile@yahoo.com,Казань
471,user_982,2019-02-28 04:20:30,500.0,0.0,-500.0,Magazinetc@yahoo.com,Ставрополь


### Задание:
Сколько раз пользователи приходили в букмекерскую контору в каждом городе?

In [5]:
#группировка по городу с подсчетом количества заходов в букмекерсую контору
sample2 = log_us.groupby('geo').user_id.count()
sample2.sort_values(ascending = False)

geo
Санкт-Петербург    115
Арзангелтск         96
Ярославль           89
Воронеж             88
Краснодар           86
Казань              66
Москва              61
Ижевск              61
Хабаровск           60
Красноярск          56
Пермь               55
Екатеринбург        49
Ставрополь          36
Тюмень              32
Name: user_id, dtype: int64

### Задание:
Во сколько раз различаются максимальное и минимальное значение средней ставки по городам?

In [6]:
#вычисление максимального и минимального значения средней ставки по всем городам
mean_bet_geo_max = log_us.query('bet > 0').groupby('geo').bet.mean().max()
mean_bet_geo_min = log_us.query('bet > 0').groupby('geo').bet.mean().min()

result = mean_bet_geo_max / mean_bet_geo_min

round(result, 2)

127.82

### Задание:
Сравнить общий процент выигрыша и проигрыша в данной букмекерской конторе

In [7]:
#сравнение % выигрыша и проигрыша
winner = round(len(log[log['net'] > 0].net) / len(log[log['bet'] > 0].net) * 100, 1)
loser = round(len(log[log['net'] < 0].net) / len(log[log['bet'] > 0].net) * 100, 1)

print('Только {}% пользователей, совершивших ставку, выигрывает. В то же время проигрывает {}% пользователей' \
      .format(winner, loser))

Только 28.2% пользователей, совершивших ставку, выигрывает. В то же время проигрывает 71.8% пользователей


### Задание:
Сколько в среднем времени проходит между появлением человека в сервисе и первой ставкой? Считать нужно только тех, кто делал ставку и у кого в файле зарегистрирован первый визит без ставки.

In [8]:
#создание списка с минимальным временем для каждого пользователя
first_time = log_us.groupby('user_id', as_index = False).agg({'time':'min'}).time.tolist()

#создание датафрейма только с минимальным временем, где первый визит без ставки
log_us2 = log_us[log_us.time.isin(first_time)]
log_us2 = log_us2[log_us2.bet == 0][['user_id', 'time']]

#список всех пользователей, у кого первый визит без ставки
first_time_us = log_us2.user_id.tolist()

#создание датафрейма с первой ненулевой ставкой по пользователям, у кого первый визит без ставки
log_us3 = log_us[log_us.user_id.isin(first_time_us)].query('bet > 0').groupby('user_id', as_index = False).agg({'time':'min'})

#объединение 2х датафреймов, вычисление среднего значения
itog = pd.merge(log_us2, log_us3, on='user_id')
(itog['time_y']-itog['time_x']).mean()

Timedelta('49 days 13:01:38.873684210')